In [210]:
import pandas as pd
import numpy as np
from scipy.stats import norm

## Data Prep ##

In [211]:
sofr_raw = pd.read_excel("hist_data.xlsm", sheet_name="SofrCurve")
aapl_raw = pd.read_excel("hist_data.xlsm", sheet_name="AAPL").set_index("Date")
msft_raw = pd.read_excel("hist_data.xlsm", sheet_name="MSFT").set_index("Date")
f_raw = pd.read_excel("hist_data.xlsm", sheet_name="F").set_index("Date")
bac_raw = pd.read_excel("hist_data.xlsm", sheet_name="BAC").set_index("Date")

In [212]:
# clean up sofr raw
sofr_raw = sofr_raw
sofr_raw.index = sofr_raw["Tenor"]
annualized_time = sofr_raw["T"]
sofr_raw.drop(columns=["T"], inplace=True)
sofr_raw.drop(columns=["Tenor"], inplace=True)
# sofr_raw.columns.name = "Date"

In [213]:
annualized_time

Tenor
1D      0.002778
1M      0.083333
2M      0.166667
3M      0.250000
6M      0.500000
9M      0.750000
1Y      1.000000
2Y      2.000000
3Y      3.000000
4Y      4.000000
5Y      5.000000
6Y      6.000000
7Y      7.000000
8Y      8.000000
9Y      9.000000
10Y    10.000000
11Y    11.000000
12Y    12.000000
13Y    13.000000
14Y    14.000000
15Y    15.000000
16Y    16.000000
17Y    17.000000
18Y    18.000000
19Y    19.000000
20Y    20.000000
25Y    25.000000
30Y    30.000000
35Y    35.000000
40Y    40.000000
Name: T, dtype: float64

In [214]:
sofr_raw

,2022-10-31,2022-11-01,2022-11-02,2022-11-03,2022-11-04,2022-11-07,2022-11-08,2022-11-09,2022-11-10,2022-11-14,...,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25,2023-10-26,2023-10-27,2023-10-30
Tenor,,,,,,,,,,,,,,,,,,,,,
1D,0.039191,0.039604,0.039948,0.040389,0.045965,0.040224,0.040339,0.040373,0.052075,0.040975,...,0.052964,0.053038,0.053050,0.053072,0.053090,0.053105,0.052981,0.053047,0.052989,0.052967
1M,0.038721,0.039023,0.039286,0.039585,0.042343,0.039258,0.039211,0.039193,0.043694,0.039174,...,0.053114,0.053102,0.053086,0.053046,0.053077,0.053084,0.053057,0.053089,0.053040,0.053053
2M,0.038670,0.038886,0.039100,0.039350,0.038795,0.039460,0.039420,0.039483,0.038786,0.039732,...,0.053374,0.053284,0.053226,0.053128,0.053191,0.053202,0.053259,0.053218,0.053185,0.053265
3M,0.040536,0.040725,0.040852,0.041154,0.040611,0.041424,0.041406,0.041445,0.040867,0.041627,...,0.053666,0.053561,0.053454,0.053323,0.053404,0.053424,0.053490,0.053414,0.053368,0.053476
6M,0.044577,0.044849,0.044884,0.045281,0.045212,0.045455,0.045334,0.045299,0.044951,0.044918,...,0.054078,0.053989,0.053758,0.053546,0.053638,0.053690,0.053749,0.053567,0.053486,0.053612
9M,0.046004,0.046448,0.046580,0.047107,0.046752,0.047189,0.046941,0.046883,0.045754,0.046142,...,0.053831,0.053753,0.053427,0.053126,0.053211,0.053293,0.053393,0.053089,0.052991,0.053120
1Y,0.046449,0.046970,0.047203,0.047894,0.047500,0.047929,0.047589,0.047479,0.045989,0.046469,...,0.053225,0.053163,0.052750,0.052348,0.052389,0.052503,0.052653,0.052243,0.052115,0.052245
2Y,0.044583,0.045022,0.045496,0.046594,0.046097,0.046633,0.046046,0.045585,0.042912,0.043394,...,0.049497,0.049611,0.049204,0.048366,0.048027,0.048399,0.048791,0.048044,0.047758,0.047904
3Y,0.042002,0.042344,0.042749,0.043833,0.043385,0.043884,0.043209,0.042645,0.039631,0.039989,...,0.046978,0.047304,0.047169,0.046151,0.045582,0.045999,0.046595,0.045645,0.045284,0.045429


In [215]:
aapl_raw.rename(columns={"Adj Close": "adj_close_aapl"}, inplace=True)
msft_raw.rename(columns={"Adj Close": "adj_close_msft"}, inplace=True)
f_raw.rename(columns={"Adj Close": "adj_close_f"}, inplace=True)
bac_raw.rename(columns={"Adj Close": "adj_close_bac"}, inplace=True)

In [216]:
aapl_raw

,adj_close_aapl
Date,
2022-10-31,152.041122
2022-11-01,149.373917
2022-11-02,143.801514
2022-11-03,137.703613
2022-11-04,137.435455
...,...
2023-10-24,172.991058
2023-10-25,170.657135
2023-10-26,166.458023


In [217]:
aapl_diff = aapl_raw.pct_change()
msft_diff = msft_raw.pct_change()
f_diff = f_raw.pct_change()
bac_diff = bac_raw.pct_change()

In [218]:
msft_diff

,adj_close_msft
Date,
2022-10-31,NaN
2022-11-01,-0.017059
2022-11-02,-0.035368
2022-11-03,-0.026579
2022-11-04,0.033326
...,...
2023-10-24,0.003674
2023-10-25,0.030678
2023-10-26,-0.037514


In [219]:
# combine df?
stock_return = pd.concat([aapl_diff, msft_diff, f_diff, bac_diff], axis=1)
stock_return.dropna(inplace=True)
stock_return

,adj_close_aapl,adj_close_msft,adj_close_f,adj_close_bac
Date,,,,
2022-11-01,-0.017543,-0.017059,0.002244,0.004439
2022-11-02,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04,-0.001947,0.033326,0.018854,0.025077
2022-11-07,0.003902,0.029270,0.014064,0.005980
...,...,...,...,...
2023-10-24,0.002543,0.003674,-0.007840,-0.003911
2023-10-25,-0.013492,0.030678,0.013169,0.003141
2023-10-26,-0.024606,-0.037514,-0.016464,0.022309


## Moddeling the Joint Distribution of the Daily Risk Factor Changes ##
* For each risk factor, specify how you want to model their changes. As I suggested, one
way is to model the absolute daily changes in the zero rate risk factors and model the
daily relative returns of the stock prices.
* Once the above is specified, follow the approach of each VaR model (parametric/Monte
Carlo/historical) to model the joint distribution of the daily risk factor changes.

For each risk factor, you would need to use the 1-year historical data provided to compute their
daily changes over the 1-year historical period. Using the historical daily changes of the risk
factors, one can then determine the joint distribution of 1-day risk factor changes under each of
the 3 VaR models

* Parametric VaR. The 1-day risk factor changes are assumed to be multivariate-normally
distributed. One can simply estimate the means and covariance matrix of the multivariate
normal distribution using the sample means and convariance matrix calculated from the
historical daily changes.
* Monte Carlo VaR. The 1-day risk factor changes are also assumed to be
multivariate-normally distributed. Hence, one simply follow the same appproach as the
parametric VaR to estimate the means and covariance matrix of the multivariate normal
distribution.
* Historical VaR. The joint distribution of the 1-day risk factor changes is simply given by
the empirical distribution of the 1-year historical sample of daily risk factor changes.
Dr. Tony Wong 4 / 21


## Stock VaR - Parametric ##

This is kinda following the lecture notes. Use the percentage diff

In [220]:
# params
stock_return_mean = stock_return.mean()
stock_return_cov = stock_return.cov()
stock_w = 1e6 * np.ones(4)

Prof Tony made a typo in the lecture notes, recall from QF6000 asset pricing
* portfolio mean return = w`R
* portfolio variance = w`Vw

Where w = weights of the portfolio, V is return covariance matrix, R is the return matrix

In [221]:
# my instinct was to use np.dot, but this is easier to read & understand
stock_port_mean = np.matmul(stock_w, stock_return_mean)
stock_port_var = np.matmul(np.matmul(stock_w, stock_return_cov), stock_w)

In [222]:
z_value = norm.ppf(0.05)

Z-value formula adapted to our convention for VaR calculation is
$$z = \frac{J-\mu}{\sigma}$$
VaR is deined as
$$ VaR_{\alpha,h} = |J|$$
$$ P(L \leq J) = \alpha $$
The project wants one-tail 95% confidence, so that means $$\alpha = 0.05$$
That means
$$J = |\mu + \sigma \cdot z_{\alpha=0.05}|

In [223]:
var_stock = abs(stock_port_mean + z_value * np.sqrt(stock_port_var))
var_stock

87730.49630401183

## Swap VaR - Parametric ##


Begin with calculating the PV01, we will find the price of the option at T = 0

In [224]:
fixed_rate = 4.2 / 100
one_bee_pee = 0.01 / 100
swap_notional = 100_000_000

In [225]:
start_df = pd.concat([annualized_time, sofr_raw[sofr_raw.columns[-1]]], axis=1)
start_df["disc_factor"] = np.exp(-start_df["T"] * start_df[start_df.columns[1]])

In [226]:
pv_fix = (
    start_df.loc["1Y", "disc_factor"]
    + start_df.loc["2Y", "disc_factor"]
    + start_df.loc["3Y", "disc_factor"]
    + start_df.loc["4Y", "disc_factor"]
    + start_df.loc["5Y", "disc_factor"]
    + start_df.loc["6Y", "disc_factor"]
    + start_df.loc["7Y", "disc_factor"]
    + start_df.loc["8Y", "disc_factor"]
    + start_df.loc["9Y", "disc_factor"]
    + start_df.loc["10Y", "disc_factor"]
) * fixed_rate
pv_float = 1 - start_df.loc["10Y", "disc_factor"]
pv_total = pv_float - pv_fix

In [227]:
print(f" {pv_float} - {pv_fix} = {pv_total}")
print("ok lah times notional")
print(pv_total * swap_notional)

 0.3561126525177397 - 0.3316836325192765 = 0.024429019998463197
ok lah times notional
2442901.9998463197


we tickle each of the zero rates by 1bp

In [228]:
start_df_tickled = start_df.iloc[6:16,].copy()

In [229]:
start_df_tickled.drop(columns=['disc_factor'],inplace=True)
tickled_pv = []

In [230]:
# this is highly inneficient but it works so stop complaining
for tenor in start_df_tickled.index:
    # add one bee pee
    start_df_tickled.loc[tenor, start_df_tickled.columns[1]] = (
        start_df_tickled.loc[tenor, start_df_tickled.columns[1]] + one_bee_pee
    )

    col_name = f"disc_factor_{tenor}"
    start_df_tickled[col_name] = np.exp(
        -start_df_tickled["T"] * start_df_tickled[start_df_tickled.columns[1]]
    )
    #col_name = start_df_tickled[start_df_tickled.columns[-1]]
    # lazy fkr tyna copy paste stuff
    temp_pv_fix = fixed_rate * start_df_tickled[start_df_tickled.columns[-1]].sum()
    temp_pv_float = 1 - start_df_tickled.loc["10Y", col_name]
    temp_pv_total = temp_pv_float - temp_pv_fix
    tickled_pv.append(temp_pv_total)
    # revert back
    start_df_tickled.loc[tenor, start_df_tickled.columns[1]] = (
        start_df_tickled.loc[tenor, start_df_tickled.columns[1]] - one_bee_pee
    )

In [231]:
start_df_tickled

,T,2023-10-30 00:00:00,disc_factor_1Y,disc_factor_2Y,disc_factor_3Y,disc_factor_4Y,disc_factor_5Y,disc_factor_6Y,disc_factor_7Y,disc_factor_8Y,disc_factor_9Y,disc_factor_10Y
Tenor,,,,,,,,,,,,
1Y,1.0,0.052245,0.949002,0.949097,0.949097,0.949097,0.949097,0.949097,0.949097,0.949097,0.949097,0.949097
2Y,2.0,0.047904,0.908638,0.908456,0.908638,0.908638,0.908638,0.908638,0.908638,0.908638,0.908638,0.908638
3Y,3.0,0.045429,0.872591,0.872591,0.872330,0.872591,0.872591,0.872591,0.872591,0.872591,0.872591,0.872591
4Y,4.0,0.044345,0.837462,0.837462,0.837462,0.837127,0.837462,0.837462,0.837462,0.837462,0.837462,0.837462
5Y,5.0,0.043928,0.802808,0.802808,0.802808,0.802808,0.802407,0.802808,0.802808,0.802808,0.802808,0.802808
6Y,6.0,0.043794,0.768923,0.768923,0.768923,0.768923,0.768923,0.768462,0.768923,0.768923,0.768923,0.768923
7Y,7.0,0.043779,0.736051,0.736051,0.736051,0.736051,0.736051,0.736051,0.735536,0.736051,0.736051,0.736051
8Y,8.0,0.043828,0.704247,0.704247,0.704247,0.704247,0.704247,0.704247,0.704247,0.703684,0.704247,0.704247
9Y,9.0,0.043915,0.673524,0.673524,0.673524,0.673524,0.673524,0.673524,0.673524,0.673524,0.672918,0.673524


In [232]:
df_tickled = pd.DataFrame(tickled_pv,index=start_df_tickled.index)
df_tickled

,0
Tenor,
1Y,0.024433
2Y,0.024437
3Y,0.024440
4Y,0.024443
5Y,0.024446
6Y,0.024448
7Y,0.024451
8Y,0.024453
9Y,0.024454


In [233]:
pv01 =  df_tickled - pv_total
pv01_notional = pv01*swap_notional

In [234]:
sliced_sofr_raw = sofr_raw.loc[sofr_raw.index.isin(pv01.index)]
sliced_sofr_raw_diff = sliced_sofr_raw.diff(axis=1)
sliced_sofr_raw_diff = sliced_sofr_raw_diff.T
sliced_sofr_raw_diff.dropna(inplace=True)
sliced_sofr_raw_diff

Tenor,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y
2022-11-01,0.000521,0.000439,0.000342,0.000296,0.000225,0.000124,0.000029,-0.000043,-9.724439e-05,-0.000144
2022-11-02,0.000234,0.000474,0.000404,0.000254,0.000153,0.000118,0.000109,0.000099,8.994989e-05,0.000091
2022-11-03,0.000690,0.001098,0.001085,0.000957,0.000832,0.000761,0.000704,0.000624,5.446595e-04,0.000495
2022-11-04,-0.000394,-0.000497,-0.000448,-0.000322,-0.000204,-0.000116,-0.000038,0.000045,1.258218e-04,0.000192
2022-11-07,0.000429,0.000536,0.000499,0.000413,0.000332,0.000313,0.000328,0.000342,3.452884e-04,0.000339
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000114,0.000371,0.000417,0.000343,0.000257,0.000165,0.000083,0.000032,2.757643e-07,-0.000027
2023-10-25,0.000150,0.000393,0.000596,0.000743,0.000894,0.000981,0.001022,0.001051,1.077486e-03,0.001100
2023-10-26,-0.000410,-0.000747,-0.000950,-0.001055,-0.001114,-0.001124,-0.001102,-0.001073,-1.046232e-03,-0.001028
2023-10-27,-0.000129,-0.000286,-0.000361,-0.000339,-0.000324,-0.000257,-0.000166,-0.000093,-3.667796e-05,0.000011


In [235]:
# params
swap_diff_mean = sliced_sofr_raw_diff.mean()
swap_diff_cov = sliced_sofr_raw_diff.cov()


In [236]:
np_pv01_notional = pv01_notional.to_numpy().flatten()*10000
np_pv01_notional

array([3.98600677e+06, 7.63179678e+06, 1.09930011e+07, 1.40665536e+07,
       1.68547510e+07, 1.93710561e+07, 2.16323359e+07, 2.36532276e+07,
       2.54477691e+07, 6.70595263e+08])

In [237]:
# my instinct was to use np.dot, but this is easier to read & understand
swap_port_mean = np.matmul(np_pv01_notional, swap_diff_mean)
swap_port_var = np.matmul(np.matmul(np_pv01_notional, swap_diff_cov), np_pv01_notional)

In [238]:
swap_port_mean

20927.251406355153

In [239]:
swap_port_var

336264178565.9113

In [240]:
var_swap = abs(swap_port_mean + z_value * np.sqrt(swap_port_var))
var_swap

932895.2410762728